In [1]:
import numpy as np
from sklearn.mixture import GaussianMixture
from scipy.stats import multivariate_normal
from sklearn.utils import shuffle

import random

## Version 1

In [2]:
class IncrementalEMGMM:
    def __init__(self, n_components, n_dims):
        """
        Initialize the Incremental EM for Gaussian Mixture Model.

        Parameters:
        - n_components: int, number of Gaussian components
        - n_dims: int, dimensionality of the data
        """
        self.n_components = n_components
        self.n_dims = n_dims
        self.weights = np.ones(n_components) / n_components  # Mixing weights
        self.means = np.random.rand(n_components, n_dims)    # Means of the Gaussians
        self.covariances = np.array([np.eye(n_dims) for _ in range(n_components)])  # Covariances
        self.total_samples = 0  # Total number of samples processed

    def e_step(self, X):
        """
        E-step: Compute responsibilities for each data point.

        Parameters:
        - X: ndarray, shape (batch_size, n_dims), batch of data

        Returns:
        - responsibilities: ndarray, shape (batch_size, n_components)
        """
        batch_size = X.shape[0]
        responsibilities = np.zeros((batch_size, self.n_components))

        # Compute the probability of each data point under each Gaussian component
        for k in range(self.n_components):
            rv = multivariate_normal(mean=self.means[k], cov=self.covariances[k])
            responsibilities[:, k] = self.weights[k] * rv.pdf(X)

        # Normalize the responsibilities across all components
        responsibilities_sum = responsibilities.sum(axis=1)[:, np.newaxis]
        responsibilities /= responsibilities_sum

        return responsibilities

    def m_step(self, X, responsibilities):
        """
        M-step: Update model parameters using responsibilities.

        Parameters:
        - X: ndarray, shape (batch_size, n_dims), batch of data
        - responsibilities: ndarray, shape (batch_size, n_components)
        """
        batch_size = X.shape[0]
        Nk = responsibilities.sum(axis=0)  # Effective number of samples for each component

        # Update weights
        self.weights = ((self.total_samples * self.weights) + Nk) / (self.total_samples + batch_size)

        # Update means and covariances incrementally
        for k in range(self.n_components):
            resp = responsibilities[:, k][:, np.newaxis]
            sum_resp = resp.sum()

            # Update means
            mean_update = (resp * X).sum(axis=0)
            self.means[k] = ((self.total_samples * self.means[k]) + mean_update) / (self.total_samples + sum_resp)

            # Update covariances
            diff = X - self.means[k]
            cov_update = np.dot((resp * diff).T, diff)
            self.covariances[k] = ((self.total_samples * self.covariances[k]) + cov_update) / (self.total_samples + sum_resp)

        self.total_samples += batch_size

    def fit(self, X_batches, n_epochs=1):
        """
        Fit the model to the data incrementally.

        Parameters:
        - X_batches: list of ndarrays, each ndarray is a batch of data
        - n_epochs: int, number of times to iterate over the batches
        """
        for epoch in range(n_epochs):
            for X in X_batches:
                responsibilities = self.e_step(X)
                self.m_step(X, responsibilities)

    def predict_proba(self, X):
        """
        Compute the posterior probabilities for each component given the data.

        Parameters:
        - X: ndarray, shape (n_samples, n_dims), input data

        Returns:
        - probabilities: ndarray, shape (n_samples, n_components)
        """
        return self.e_step(X)

    def predict(self, X):
        """
        Assign each data point to the component with the highest posterior probability.

        Parameters:
        - X: ndarray, shape (n_samples, n_dims), input data

        Returns:
        - labels: ndarray, shape (n_samples,), component assignments
        """
        responsibilities = self.e_step(X)
        return np.argmax(responsibilities, axis=1)


In [3]:
# Example usage:
if __name__ == "__main__":
    # Generate synthetic data
    np.random.seed(42)
    n_dims = 2
    n_components = 3
    n_samples = 1000

    true_means = np.array([[0, 0], [5, 5], [0, 5]])
    true_covs = np.array([np.eye(2), np.eye(2), np.eye(2)])
    true_weights = np.array([0.3, 0.5, 0.2])

    # Generate samples from the true GMM
    X = np.vstack([
        np.random.multivariate_normal(true_means[k], true_covs[k], size=int(true_weights[k]*n_samples))
        for k in range(n_components)
    ])

    # Shuffle the data
    np.random.shuffle(X)

    # Split data into mini-batches
    batch_size = 100
    X_batches = [X[i:i + batch_size] for i in range(0, n_samples, batch_size)]

    # Initialize and fit the incremental EM GMM
    incremental_gmm = IncrementalEMGMM(n_components=n_components, n_dims=n_dims)
    incremental_gmm.fit(X_batches, n_epochs=5)

    # Predict component assignments
    labels = incremental_gmm.predict(X)

    # Print the estimated means
    print("Estimated Means:")
    print(incremental_gmm.means)

Estimated Means:
[[0.95493457 2.69975487]
 [4.54172057 4.55038648]
 [0.93569719 2.68721811]]


## Version 2

In [4]:
import numpy as np
from scipy.stats import multivariate_normal

class OnlineEMGMM:
    def __init__(self, n_components, n_dims, t0=10, kappa=0.6):
        """
        Initialize the Online EM for Gaussian Mixture Model.

        Parameters:
        - n_components: int, number of Gaussian components
        - n_dims: int, dimensionality of the data
        - t0: float, delay parameter for learning rate schedule
        - kappa: float, forgetting factor (0.5 < kappa <= 1)
        """
        self.n_components = n_components
        self.n_dims = n_dims
        self.t = 0  # Time step (number of samples processed)

        # Initialize parameters
        self.weights = np.ones(n_components) / n_components  # Mixing weights π_k
        self.means = np.random.randn(n_components, n_dims)   # Means μ_k
        self.covariances = np.array([np.eye(n_dims) for _ in range(n_components)])  # Covariances Σ_k

        # Initialize sufficient statistics
        self.s0 = np.zeros(n_components)  # Sum of responsibilities for each component
        self.s1 = np.zeros((n_components, n_dims))  # Sum of weighted data points
        self.s2 = np.zeros((n_components, n_dims, n_dims))  # Sum of weighted outer products

        # Learning rate schedule parameters
        self.t0 = t0
        self.kappa = kappa

    def compute_responsibilities(self, x):
        """
        E-step: Compute responsibilities for a single data point.

        Parameters:
        - x: ndarray, shape (n_dims,), a single data sample

        Returns:
        - responsibilities: ndarray, shape (n_components,), responsibilities for each component
        """
        responsibilities = np.zeros(self.n_components)
        for k in range(self.n_components):
            rv = multivariate_normal(mean=self.means[k], cov=self.covariances[k], allow_singular=True)
            responsibilities[k] = self.weights[k] * rv.pdf(x)
        total = responsibilities.sum()
        if total == 0:
            # Avoid division by zero
            responsibilities = np.ones(self.n_components) / self.n_components
        else:
            responsibilities /= total
        return responsibilities

    def update(self, x):
        """
        Update the model parameters with a single data point.

        Parameters:
        - x: ndarray, shape (n_dims,), a single data sample
        """
        self.t += 1  # Increment time step
        alpha_t = (self.t0 + self.t) ** -self.kappa  # Learning rate

        # E-step: Compute responsibilities
        gamma = self.compute_responsibilities(x)

        # M-step: Update sufficient statistics
        for k in range(self.n_components):
            # Update sufficient statistics with forgetting factor
            self.s0[k] = (1 - alpha_t) * self.s0[k] + alpha_t * gamma[k]
            self.s1[k] = (1 - alpha_t) * self.s1[k] + alpha_t * gamma[k] * x
            x_diff = x - self.means[k]
            outer_product = np.outer(x_diff, x_diff)
            self.s2[k] = (1 - alpha_t) * self.s2[k] + alpha_t * gamma[k] * outer_product

            # Update parameters
            if self.s0[k] > 1e-6:  # Avoid division by zero
                self.weights[k] = self.s0[k] / self.s0.sum()
                self.means[k] = self.s1[k] / self.s0[k]
                cov_matrix = self.s2[k] / self.s0[k]
                cov_matrix = cov_matrix + 1e-6 * np.eye(self.n_dims)  # Regularization
                self.covariances[k] = cov_matrix

    def fit(self, X_stream):
        """
        Fit the model to the data stream.

        Parameters:
        - X_stream: iterable of ndarrays, each ndarray is a data sample
        """
        for x in X_stream:
            self.update(x)

    def predict_proba(self, x):
        """
        Compute the posterior probabilities for each component given a data point.

        Parameters:
        - x: ndarray, shape (n_dims,), a single data sample

        Returns:
        - probabilities: ndarray, shape (n_components,)
        """
        return self.compute_responsibilities(x)

    def predict(self, x):
        """
        Assign a data point to the component with the highest posterior probability.

        Parameters:
        - x: ndarray, shape (n_dims,), a single data sample

        Returns:
        - label: int, component assignment
        """
        responsibilities = self.compute_responsibilities(x)
        return np.argmax(responsibilities)

# Example usage:
if __name__ == "__main__":
    # Generate synthetic data
    np.random.seed(42)
    n_dims = 2
    n_components = 3
    n_samples = 1000

    true_means = np.array([[0, 0], [5, 5], [0, 5]])
    true_covs = np.array([np.eye(2), np.eye(2), np.eye(2)])
    true_weights = np.array([0.3, 0.5, 0.2])

    # Generate samples from the true GMM
    X = np.vstack([
        np.random.multivariate_normal(true_means[k], true_covs[k], size=int(true_weights[k]*n_samples))
        for k in range(n_components)
    ])

    # Shuffle the data
    np.random.shuffle(X)

    # Initialize the online EM GMM
    online_gmm = OnlineEMGMM(n_components=n_components, n_dims=n_dims)

    # Simulate data arriving one by one
    for x in X:
        online_gmm.update(x)

    # After processing all data, print the estimated means
    print("Estimated Means:")
    print(online_gmm.means)

    # Predict component assignments for all data
    labels = [online_gmm.predict(x) for x in X]

    # Optionally, compare the estimated parameters with the true parameters
    print("\nTrue Means:")
    print(true_means)


Estimated Means:
[[ 4.75037457  4.54113817]
 [-0.00788832  2.03898053]
 [ 5.11664584  5.18037287]]

True Means:
[[0 0]
 [5 5]
 [0 5]]


## Version 3

In [5]:
import numpy as np
from scipy.stats import multivariate_normal

class OnlineEMGMM:
    def __init__(self, n_components, n_dims, learning_rate=0.1):
        """
        Initialize the Online EM for Gaussian Mixture Model.

        Parameters:
        - n_components: int, number of Gaussian components
        - n_dims: int, dimensionality of the data
        - learning_rate: float, controls the rate of updating
        """
        self.n_components = n_components
        self.n_dims = n_dims
        self.learning_rate = learning_rate

        # Initialize parameters
        self.weights = np.ones(n_components) / n_components  # Mixing weights
        self.means = np.random.rand(n_components, n_dims)    # Means of the Gaussians
        self.covariances = np.array([np.eye(n_dims) for _ in range(n_components)])  # Covariances

    def e_step(self, x):
        """
        E-step: Compute responsibilities for a single data point.

        Parameters:
        - x: ndarray, shape (n_dims,), a single data point

        Returns:
        - responsibilities: ndarray, shape (n_components,)
        """
        responsibilities = np.zeros(self.n_components)

        # Compute the probability of the data point under each Gaussian component
        for k in range(self.n_components):
            rv = multivariate_normal(mean=self.means[k], cov=self.covariances[k])
            responsibilities[k] = self.weights[k] * rv.pdf(x)

        # Normalize responsibilities
        responsibilities /= responsibilities.sum()

        return responsibilities

    def m_step(self, x, responsibilities):
        """
        M-step: Update model parameters for a single data point.

        Parameters:
        - x: ndarray, shape (n_dims,), a single data point
        - responsibilities: ndarray, shape (n_components,), responsibilities for the data point
        """
        for k in range(self.n_components):
            # Learning rate
            lr = self.learning_rate * responsibilities[k]

            # Update weights
            self.weights[k] = (1 - self.learning_rate) * self.weights[k] + self.learning_rate * responsibilities[k]

            # Update means
            self.means[k] = (1 - lr) * self.means[k] + lr * x

            # Update covariances
            diff = (x - self.means[k]).reshape(-1, 1)
            self.covariances[k] = (1 - lr) * self.covariances[k] + lr * np.dot(diff, diff.T)

        # Normalize weights
        self.weights /= self.weights.sum()

    def update(self, x):
        """
        Process a single data point and update the model parameters.

        Parameters:
        - x: ndarray, shape (n_dims,), a single data point
        """
        responsibilities = self.e_step(x)
        self.m_step(x, responsibilities)

    def predict_proba(self, x):
        """
        Compute the posterior probabilities for a single data point.

        Parameters:
        - x: ndarray, shape (n_dims,), a single data point

        Returns:
        - responsibilities: ndarray, shape (n_components,)
        """
        return self.e_step(x)

    def predict(self, x):
        """
        Assign the data point to the component with the highest posterior probability.

        Parameters:
        - x: ndarray, shape (n_dims,), a single data point

        Returns:
        - label: int, the component assignment
        """
        responsibilities = self.e_step(x)
        return np.argmax(responsibilities)

# Example usage
if __name__ == "__main__":
    # Generate synthetic data
    np.random.seed(42)
    n_dims = 2
    n_components = 3
    n_samples = 1000

    true_means = np.array([[0, 0], [5, 5], [0, 5]])
    true_covs = np.array([np.eye(2), np.eye(2), np.eye(2)])
    true_weights = np.array([0.3, 0.5, 0.2])

    # Generate samples from the true GMM
    X = np.vstack([
        np.random.multivariate_normal(true_means[k], true_covs[k], size=int(true_weights[k] * n_samples))
        for k in range(n_components)
    ])

    # Shuffle the data
    np.random.shuffle(X)

    # Initialize the online EM GMM
    online_gmm = OnlineEMGMM(n_components=n_components, n_dims=n_dims, learning_rate=0.05)

    # Process each data point one by one
    for x in X:
        online_gmm.update(x)

    # Print the estimated means
    print("Estimated Means:")
    print(online_gmm.means)


Estimated Means:
[[ 0.09730099 -0.04656867]
 [ 3.41813348  5.11451274]
 [ 0.09210458 -0.05114168]]


## Version 4

In [6]:
import numpy as np
from scipy.stats import multivariate_normal

class IncrementalEMGMM:
    def __init__(self, n_components, n_dims):
        """
        Initialize the Incremental EM for Gaussian Mixture Model.

        Parameters:
        - n_components: int, number of Gaussian components
        - n_dims: int, dimensionality of the data
        """
        self.n_components = n_components
        self.n_dims = n_dims

        # Initialize parameters θ(µ)
        self.weights = np.ones(n_components) / n_components  # Mixing weights
        self.means = np.random.rand(n_components, n_dims)    # Means of the Gaussians
        self.covariances = np.array([np.eye(n_dims) for _ in range(n_components)])  # Covariances

        # Initialize s_i for all data points i
        self.s_i_list = None  # Will be initialized in fit method

        # Initialize µ = sum_i s_i (sufficient statistics)
        self.N_k = np.zeros(n_components)  # Component counts
        self.Sum_k = np.zeros((n_components, n_dims))  # Sum of x_i for each component
        self.Sum_squares_k = np.zeros((n_components, n_dims, n_dims))  # Sum of x_i x_i^T for each component

    def initialize_s_i(self, X):
        """
        Initialize s_i for each data point i.

        Parameters:
        - X: ndarray, shape (n_samples, n_dims), the data
        """
        n_samples = X.shape[0]
        self.s_i_list = []

        # Random initial responsibilities
        for i in range(n_samples):
            gamma_ik = np.random.dirichlet(np.ones(self.n_components))

            s_i_k = []
            x_i = X[i]
            for k in range(self.n_components):
                gamma = gamma_ik[k]
                s_i_k.append({
                    'gamma': gamma,
                    'gamma_x': gamma * x_i,
                    'gamma_xxT': gamma * np.outer(x_i, x_i)
                })

                # Update µ
                self.N_k[k] += gamma
                self.Sum_k[k] += gamma * x_i
                self.Sum_squares_k[k] += gamma * np.outer(x_i, x_i)

            self.s_i_list.append(s_i_k)

        # Update θ(µ)
        self.update_parameters()

    def update_parameters(self):
        """
        Update model parameters θ(µ) based on the current µ (sufficient statistics).
        """
        total_N = np.sum(self.N_k)
        self.weights = self.N_k / total_N
        self.means = self.Sum_k / self.N_k[:, np.newaxis]
        self.covariances = np.zeros((self.n_components, self.n_dims, self.n_dims))

        for k in range(self.n_components):
            diff = self.Sum_squares_k[k] / self.N_k[k] - np.outer(self.means[k], self.means[k])
            self.covariances[k] = diff + np.eye(self.n_dims) * 1e-6  # Regularization

    def fit(self, X, n_iterations=10):
        """
        Fit the model to the data incrementally.

        Parameters:
        - X: ndarray, shape (n_samples, n_dims), the data
        - n_iterations: int, number of EM iterations
        """
        n_samples = X.shape[0]

        # Initialize s_i for each data point
        self.initialize_s_i(X)

        # For each iteration
        for iteration in range(n_iterations):
            # Process data points in random order
            indices = np.random.permutation(n_samples)
            for idx in indices:
                x_i = X[idx]
                s_i_old = self.s_i_list[idx]
                s_i_new = []

                # Compute responsibilities gamma_ik = p(z=k | x_i; θ(µ))
                gamma_ik = np.zeros(self.n_components)
                for k in range(self.n_components):
                    rv = multivariate_normal(mean=self.means[k], cov=self.covariances[k])
                    gamma_ik[k] = self.weights[k] * rv.pdf(x_i)
                gamma_ik /= np.sum(gamma_ik)

                # Compute s'_i
                for k in range(self.n_components):
                    gamma = gamma_ik[k]
                    s_i_k = {
                        'gamma': gamma,
                        'gamma_x': gamma * x_i,
                        'gamma_xxT': gamma * np.outer(x_i, x_i)
                    }
                    s_i_new.append(s_i_k)

                    # Update µ ← µ + s'_i_k - s_i_old_k
                    self.N_k[k] += s_i_k['gamma'] - s_i_old[k]['gamma']
                    self.Sum_k[k] += s_i_k['gamma_x'] - s_i_old[k]['gamma_x']
                    self.Sum_squares_k[k] += s_i_k['gamma_xxT'] - s_i_old[k]['gamma_xxT']

                # Replace s_i ← s'_i
                self.s_i_list[idx] = s_i_new

            # Update parameters after each iteration
            self.update_parameters()

    def predict_proba(self, X):
        """
        Compute the posterior probabilities for each component given the data.

        Parameters:
        - X: ndarray, shape (n_samples, n_dims), input data

        Returns:
        - probabilities: ndarray, shape (n_samples, n_components)
        """
        n_samples = X.shape[0]
        responsibilities = np.zeros((n_samples, self.n_components))

        for i in range(n_samples):
            x_i = X[i]
            for k in range(self.n_components):
                rv = multivariate_normal(mean=self.means[k], cov=self.covariances[k])
                responsibilities[i, k] = self.weights[k] * rv.pdf(x_i)
            responsibilities[i, :] /= np.sum(responsibilities[i, :])

        return responsibilities

    def predict(self, X):
        """
        Assign each data point to the component with the highest posterior probability.

        Parameters:
        - X: ndarray, shape (n_samples, n_dims), input data

        Returns:
        - labels: ndarray, shape (n_samples,), component assignments
        """
        responsibilities = self.predict_proba(X)
        return np.argmax(responsibilities, axis=1)

# Example usage:
if __name__ == "__main__":
    # Generate synthetic data
    np.random.seed(42)
    n_dims = 2
    n_components = 3
    n_samples = 1000

    true_means = np.array([[0, 0], [5, 5], [0, 5]])
    true_covs = np.array([np.eye(2), np.eye(2), np.eye(2)])
    true_weights = np.array([0.3, 0.5, 0.2])

    # Generate samples from the true GMM
    X = np.vstack([
        np.random.multivariate_normal(true_means[k], true_covs[k], size=int(true_weights[k]*n_samples))
        for k in range(n_components)
    ])

    # Shuffle the data
    np.random.shuffle(X)

    # Initialize and fit the incremental EM GMM
    incremental_gmm = IncrementalEMGMM(n_components=n_components, n_dims=n_dims)
    incremental_gmm.fit(X, n_iterations=20)

    # Predict component assignments
    labels = incremental_gmm.predict(X)

    # Print the estimated means
    print("\nEstimated Means:")
    print(incremental_gmm.means)



Estimated Means:
[[ 3.03238479  2.57738491]
 [-0.04135715 -0.05512272]
 [ 3.49319566  5.17840849]]
